In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import zipfile
import io
import os
import json

In [3]:
fzips = glob("../ZipDump/*.zip")
fzips

['../ZipDump/IDCJAC0011_58161_1800.zip',
 '../ZipDump/IDCJAC0011_66037_1800.zip',
 '../ZipDump/IDCJAC0010_076031_1800.zip',
 '../ZipDump/IDCJAC0010_023034_1800.zip',
 '../ZipDump/IDCJAC0011_39326_1800.zip',
 '../ZipDump/IDCJAC0011_86282_1800.zip',
 '../ZipDump/IDCJAC0011_23034_1800.zip',
 '../ZipDump/IDCJAC0011_76031_1800.zip',
 '../ZipDump/IDCJAC0010_086282_1800.zip',
 '../ZipDump/IDCJAC0010_039326_1800.zip',
 '../ZipDump/IDCJAC0010_058161_1800.zip',
 '../ZipDump/IDCJAC0010_066037_1800.zip',
 '../ZipDump/IDCJAC0011_40764_1800.zip',
 '../ZipDump/IDCJAC0011_72160_1800.zip',
 '../ZipDump/IDCJAC0011_39128_1800.zip',
 '../ZipDump/IDCJAC0011_61078_1800.zip',
 '../ZipDump/IDCJAC0011_041529_1800.zip',
 '../ZipDump/IDCJAC0011_68241_1800.zip',
 '../ZipDump/IDCJAC0010_047048_1800.zip',
 '../ZipDump/IDCJAC0010_026021_1800.zip',
 '../ZipDump/IDCJAC0011_24584_1800.zip',
 '../ZipDump/IDCJAC0010_039128_1800.zip',
 '../ZipDump/IDCJAC0010_061078_1800.zip',
 '../ZipDump/IDCJAC0010_040764_1800.zip',
 '..

In [4]:
bomid = re.compile("station number:[ \d]*")
stname = re.compile("Station name:[ \w]*")
temptype = re.compile("[\w]{3}imum Temp")

In [5]:
%%time
alltemps = {}
for fz in fzips:
    with zipfile.ZipFile(fz) as zopen:
        ftext = [n for n in zopen.namelist() if '.txt' in n][0]
        fcsv = [n for n in zopen.namelist() if '.csv' in n][0]
        with zopen.open(ftext,'r') as infile:

            txt = io.TextIOWrapper(io.BytesIO(infile.read()))
            txt = ''.join([t for t in txt])

        outid = re.findall(bomid, txt)[0].split(':')[1][1:]
        outname = re.findall(stname, txt)[0].split(':')[1][1:]
        outname = '-'.join(o for o in outname.split() if o != 'AWS')
        mmtemp = re.findall(temptype, txt)[0][:3]

        fullname = '_'.join([outname, mmtemp + 'T', outid])
        
        zopen.extract(fcsv) ## only once off, to extract the csv to cwd
    alltemps[fcsv] = fullname
    print(fullname)
print(alltemps)

GRAFTON-AIRPORT_MinT_58161
SYDNEY-AIRPORT-AMO_MinT_66037
MILDURA-AIRPORT_MaxT_76031
ADELAIDE-AIRPORT_MaxT_23034
GLADSTONE-AIRPORT_MinT_39326
MELBOURNE-AIRPORT_MinT_86282
ADELAIDE-AIRPORT_MinT_23034
MILDURA-AIRPORT_MinT_76031
MELBOURNE-AIRPORT_MaxT_86282
GLADSTONE-AIRPORT_MaxT_39326
GRAFTON-AIRPORT_MaxT_58161
SYDNEY-AIRPORT-AMO_MaxT_66037
GOLD-COAST-SEAWAY_MinT_40764
ALBURY-AIRPORT_MinT_72160
BUNDABERG-AERO_MinT_39128
WILLIAMTOWN-RAAF_MinT_61078
TOOWOOMBA-AIRPORT_MinT_41529
ALBION-PARK_MinT_68241
BROKEN-HILL-AIRPORT_MaxT_47048
MOUNT-GAMBIER-AERO_MaxT_26021
MURRAY-BRIDGE_MinT_24584
BUNDABERG-AERO_MaxT_39128
WILLIAMTOWN-RAAF_MaxT_61078
GOLD-COAST-SEAWAY_MaxT_40764
ALBURY-AIRPORT_MaxT_72160
MOUNT-GAMBIER-AERO_MinT_26021
MURRAY-BRIDGE_MaxT_24584
TOOWOOMBA-AIRPORT_MaxT_41529
BROKEN-HILL-AIRPORT_MinT_47048
ALBION-PARK_MaxT_68241
{'IDCJAC0011_58161_1800_Data.csv': 'GRAFTON-AIRPORT_MinT_58161', 'IDCJAC0011_66037_1800_Data.csv': 'SYDNEY-AIRPORT-AMO_MinT_66037', 'IDCJAC0010_076031_1800_Data.csv':

In [6]:
for original, new in alltemps.items():
#     print(original, new)
    os.rename(original, "./base-data/" + new + ".csv")

In [7]:
relat = re.compile(r"(Latitude)(.*):(.*)")
relong = re.compile(r"(Longitude)(.*):(.*)")
restate = re.compile(r"State: (.*)")

In [8]:
%%time
infotemps = {}
for fz in fzips:
    with zipfile.ZipFile(fz) as zopen:
        ftext = [n for n in zopen.namelist() if '.txt' in n][0]
        fcsv = [n for n in zopen.namelist() if '.csv' in n][0]
        with zopen.open(ftext,'r') as infile:

            txt = io.TextIOWrapper(io.BytesIO(infile.read()))
            txt = ''.join([t for t in txt])

        outid = re.findall(bomid, txt)[0].split(':')[1][1:]
        outname = re.findall(stname, txt)[0].split(':')[1][1:]
        outname = '-'.join(o for o in outname.split() if o != 'AWS')
        mmtemp = re.findall(temptype, txt)[0][:3]

        fullname = '_'.join([outname, mmtemp + 'T', outid])
        
        lat = re.findall(relat, txt)[0][-1].strip()
        long = re.findall(relong, txt)[0][-1].strip()
        st = re.findall(restate, txt)[0].strip()
        
        infotemps[outid] = {k:v for k,v in zip(["Name", "LatLong", "State"],[outname, (lat,long), st])}
        
        
        
infotemps
    
    
    
    

CPU times: user 17.1 ms, sys: 5.63 ms, total: 22.8 ms
Wall time: 28.8 ms


In [9]:
infotemps

{'58161': {'Name': 'GRAFTON-AIRPORT',
  'LatLong': ('-29.76', '153.03'),
  'State': 'NSW'},
 '66037': {'Name': 'SYDNEY-AIRPORT-AMO',
  'LatLong': ('-33.95', '151.17'),
  'State': 'NSW'},
 '76031': {'Name': 'MILDURA-AIRPORT',
  'LatLong': ('-34.24', '142.09'),
  'State': 'VIC'},
 '23034': {'Name': 'ADELAIDE-AIRPORT',
  'LatLong': ('-34.95', '138.52'),
  'State': 'SA'},
 '39326': {'Name': 'GLADSTONE-AIRPORT',
  'LatLong': ('-23.87', '151.22'),
  'State': 'QLD'},
 '86282': {'Name': 'MELBOURNE-AIRPORT',
  'LatLong': ('-37.67', '144.83'),
  'State': 'VIC'},
 '40764': {'Name': 'GOLD-COAST-SEAWAY',
  'LatLong': ('-27.94', '153.43'),
  'State': 'QLD'},
 '72160': {'Name': 'ALBURY-AIRPORT',
  'LatLong': ('-36.07', '146.95'),
  'State': 'NSW'},
 '39128': {'Name': 'BUNDABERG-AERO',
  'LatLong': ('-24.91', '152.32'),
  'State': 'QLD'},
 '61078': {'Name': 'WILLIAMTOWN-RAAF',
  'LatLong': ('-32.79', '151.84'),
  'State': 'NSW'},
 '41529': {'Name': 'TOOWOOMBA-AIRPORT',
  'LatLong': ('-27.54', '151.91'

In [10]:
oldnotes = glob("../*/*.txt") + glob("./base-data/*.txt")
oldnotes

['../ZipDump/4bda4fce428abc201ce825db83b422085da007.txt',
 '../ZipDump/d206dc7de9fa9e09fbad36865c973d2dd82ad3.txt',
 '../ZipDump/501a5be88024d1e2a48e9de0f07c3a4cd9f6b2.txt',
 '../ZipDump/c628dc992aa66ffec97c0b63ce77a43d30b0e1.txt',
 '../ZipDump/97f5df45211225d087e43ad14598a6c1ca5a7e.txt',
 '../ZipDump/2a776952e29ec63ebe25c0e4bfdf8582d21d35.txt',
 '../ZipDump/36d0c5c73312bee8a74e352a52aec06ce72af6.txt',
 '../ZipDump/4e808b9bff58579814f47518386773f6112dcf.txt',
 '../ZipDump/34ddcdadfd269ede55b93206334d13422e4840.txt',
 '../ZipDump/b24064c77449e99ec3b49abe3452b6fa6ad609.txt',
 '../ZipDump/0e79e694e20578e66b9c59499d7f671d9cbd93.txt',
 '../ZipDump/37d153be3c7df84279704d9a322e734df488c0.txt',
 '../ZipDump/05ad874d09aba39244ed58c95f88a7a4157e42.txt',
 '../ZipDump/816c5057b0205ad6741cb44eb0bdc28b314e64.txt',
 '../ZipDump/fe0db15a6233192e72b6f92e612ae34499c92f.txt',
 '../ZipDump/019d4ef829b0a5750eb46d2df1225313235f28.txt',
 '../ZipDump/a15bd44552c4b99633f4c53b0b8ce08344e890.txt',
 '../ZipDump/d

In [11]:
for note in oldnotes:
    with open(note, 'r') as infile:
        txt = infile.read()
    outid = re.findall(bomid, txt)[0].split(':')[1][1:]
    outname = re.findall(stname, txt)[0].split(':')[1][1:]
    outname = '-'.join(o for o in outname.split() if o != 'AWS')
    mmtemp = re.findall(temptype, txt)[0][:3]

    fullname = '_'.join([outname, mmtemp + 'T', outid])

    lat = re.findall(relat, txt)[0][-1].strip()
    long = re.findall(relong, txt)[0][-1].strip()
    st = re.findall(restate, txt)[0].strip()

    infotemps[outid] = {k:v for k,v in zip(["Name", "LatLong", "State"],[outname, (lat,long), st])}
    
infotemps

{'58161': {'Name': 'GRAFTON-AIRPORT',
  'LatLong': ('-29.76', '153.03'),
  'State': 'NSW'},
 '66037': {'Name': 'SYDNEY-AIRPORT-AMO',
  'LatLong': ('-33.95', '151.17'),
  'State': 'NSW'},
 '76031': {'Name': 'MILDURA-AIRPORT',
  'LatLong': ('-34.24', '142.09'),
  'State': 'VIC'},
 '23034': {'Name': 'ADELAIDE-AIRPORT',
  'LatLong': ('-34.95', '138.52'),
  'State': 'SA'},
 '39326': {'Name': 'GLADSTONE-AIRPORT',
  'LatLong': ('-23.87', '151.22'),
  'State': 'QLD'},
 '86282': {'Name': 'MELBOURNE-AIRPORT',
  'LatLong': ('-37.67', '144.83'),
  'State': 'VIC'},
 '40764': {'Name': 'GOLD-COAST-SEAWAY',
  'LatLong': ('-27.94', '153.43'),
  'State': 'QLD'},
 '72160': {'Name': 'ALBURY-AIRPORT',
  'LatLong': ('-36.07', '146.95'),
  'State': 'NSW'},
 '39128': {'Name': 'BUNDABERG-AERO',
  'LatLong': ('-24.91', '152.32'),
  'State': 'QLD'},
 '61078': {'Name': 'WILLIAMTOWN-RAAF',
  'LatLong': ('-32.79', '151.84'),
  'State': 'NSW'},
 '41529': {'Name': 'TOOWOOMBA-AIRPORT',
  'LatLong': ('-27.54', '151.91'

In [12]:
with open('station-data.json', 'w') as outfile:
    json.dump(infotemps, outfile)